In [93]:
import pandas as pd
from hierarchicalforecast.utils import aggregate
from statsforecast import StatsForecast
from statsforecast.models import AutoARIMA
from hierarchicalforecast.core import HierarchicalReconciliation
from hierarchicalforecast.methods import BottomUp
import matplotlib.pyplot as plt

In [42]:
def cargar_datos():
    # Cargar datos, manejar metadatos y formato de fechas
    df = pd.read_csv(
        'data/Consulta_20250224-120620223.csv',
        skiprows=8,
        dtype=str,
        encoding='latin1'
    ).reset_index()
    headers = df.iloc[0].tolist()
    df = df[1:]
    df.columns = headers

    # Filtrar filas con fechas válidas
    mask = df['Título'].str.match(r'\d{2}/\d{2}/\d{4}', na=False)
    df = df[mask].copy()

    # Convertir fechas a fin de trimestre
    df['ds'] = pd.to_datetime(df['Título'], format='%d/%m/%Y') + pd.offsets.QuarterEnd()

    # Pivotear datos a formato largo
    value_vars = headers[1:]    # Excluir fecha
    df_melted = df.melt(
        id_vars=['Título', 'ds'],
        value_vars=value_vars,
        var_name='unique_id',
        value_name='y'
    )
    df_melted['y'] = pd.to_numeric(df_melted['y'], errors='coerce')
    df_melted = df_melted.dropna(subset=['y'])

    return df_melted[['unique_id', 'ds', 'y']]

df_procesado = cargar_datos()

display(df_procesado)

,unique_id,ds,y
3,Financiamiento bancario y principales fuentes ...,1994-12-31,51133.0
4,Financiamiento bancario y principales fuentes ...,1995-03-31,0.0
5,Financiamiento bancario y principales fuentes ...,1995-06-30,0.0
6,Financiamiento bancario y principales fuentes ...,1995-09-30,0.0
7,Financiamiento bancario y principales fuentes ...,1995-12-31,46988.6
...,...,...,...
8851,Financiamiento total al sector privado no fina...,2023-09-30,751391.1
8852,Financiamiento total al sector privado no fina...,2023-12-31,807119.2
8853,Financiamiento total al sector privado no fina...,2024-03-31,650324.2
8854,Financiamiento total al sector privado no fina...,2024-06-30,675546.1


In [ ]:
def procesar_datos(df_procesado):
    df_procesado['unique_id'] = df_procesado['unique_id'].str.replace('/', '_')
    # Determinar niveles jerárquicos basados en comas en unique_id
    split_ids = df_procesado['unique_id'].str.split(', ', expand=True)
    max_depth = split_ids.shape[1]

    # Crear columnas de nivel
    level_cols = [f'level{i + 1}' for i in range(max_depth)]
    for i, col in enumerate(level_cols):
        df_procesado[col] = split_ids[i]
    
    df_procesado = df_procesado.fillna('end')
    
    # Definir agregación
    spec = []
    for depth in range(1, max_depth + 1):
        spec.append([f'level{i}' for i in range(1, depth + 1)])

    # Agregar jerarquía
    Y_hier_df, S_df, tags = aggregate(df_procesado, spec=spec)
    
    return Y_hier_df, S_df, tags

Y_hier_df, S_df, tags = procesar_datos(df_procesado)

In [118]:
def pronosticar_y_reconciliar(Y_hier_df):
    # Entrenar modelos y reconciliar
    sf = StatsForecast(
        models=[AutoARIMA(season_length=4)],  # Trimestral
        freq='Q',
        n_jobs=-1
    )

    sf.fit(Y_hier_df)
    forecast = sf.predict(h=30)  # Pronóstico para 2 años
    reconciler = HierarchicalReconciliation(reconcilers=[BottomUp()])
    reconciled = reconciler.reconcile(
        Y_hat_df=forecast,
        Y_df=Y_hier_df,
        S=S_df,
        tags=tags
    )
    
    return forecast, reconciled

forecast, reconciled = pronosticar_y_reconciliar(Y_hier_df)

c:\Users\seelro06\OneDrive - Arca Continental S.A.B. de C.V\Documentos\uni\del-negociante\.venv\Lib\site-packages\utilsforecast\processing.py:384: FutureWarning: 'Q' is deprecated and will be removed in a future version, please use 'QE' instead.
  freq = pd.tseries.frequencies.to_offset(freq)
c:\Users\seelro06\OneDrive - Arca Continental S.A.B. de C.V\Documentos\uni\del-negociante\.venv\Lib\site-packages\utilsforecast\processing.py:440: FutureWarning: 'Q' is deprecated and will be removed in a future version, please use 'QE' instead.
  freq = pd.tseries.frequencies.to_offset(freq)


In [119]:
def calcular_participacion(reconciled):
    # Calcular participación de mercado
    df_wide = reconciled.set_index(['ds', 'unique_id'])['AutoARIMA/BottomUp'].unstack()
    total = df_wide.sum(axis=1)
    participacion = df_wide.div(total, axis=0)
    return participacion

participacion = calcular_participacion(reconciled)

display(participacion)

unique_id,Financiamiento bancario y principales fuentes altern endeud sec privado del país,Financiamiento bancario y principales fuentes altern endeud sec privado del país /No bancarios,Financiamiento bancario y principales fuentes altern endeud sec privado del país /No bancarios/Clasif p_ramas actividad,Financiamiento bancario y principales fuentes altern endeud sec privado del país /No bancarios/Clasif p_ramas actividad/financ a emp y pers físicas con actividad empresarial,Financiamiento bancario y principales fuentes altern endeud sec privado del país /No bancarios/Clasif p_ramas actividad/financ a emp y pers físicas con actividad empresarial/No banc,Financiamiento bancario y principales fuentes altern endeud sec privado del país /No bancarios/Clasif p_ramas actividad/financ a emp y pers físicas con actividad empresarial/No banc/textiles prend vestir ind cuero,Financiamiento bancario y principales fuentes altern endeud sec privado del país /No bancarios/Clasif p_ramas actividad/financ a emp y pers físicas con actividad empresarial/No banc/textiles prend vestir ind cuero/De Intermediarios Financieros del País,Financiamiento bancario y principales fuentes altern endeud sec privado del país /No bancarios/Clasif p_ramas actividad/financ a emp y pers físicas con actividad empresarial/No banc/textiles prend vestir ind cuero/Otras Fuentes Alternativas de Financiamiento del País,Financiamiento bancario y principales fuentes altern endeud sec privado del país /No bancarios/Clasif p_ramas actividad/financ a emp y pers físicas con actividad empresarial/No banc/textiles prend vestir ind cuero/end,Financiamiento bancario y principales fuentes altern endeud sec privado del país /No bancarios/del extranjero,...,Financiamiento total al sector privado no financiero/Financiamiento total/interno/Emision interna de instrumentos de deuda/end/end,Financiamiento total al sector privado no financiero/Financiamiento total/interno/Emision interna de instrumentos de deuda/end/end/end,Financiamiento total al sector privado no financiero/Financiamiento total/interno/Proveedores de Emisoras,Financiamiento total al sector privado no financiero/Financiamiento total/interno/Proveedores de Emisoras/end,Financiamiento total al sector privado no financiero/Financiamiento total/interno/Proveedores de Emisoras/end/end,Financiamiento total al sector privado no financiero/Financiamiento total/interno/Proveedores de Emisoras/end/end/end,Financiamiento total al sector privado no financiero/Financiamiento total/interno/Total de endeudamiento no bancario de emisoras,Financiamiento total al sector privado no financiero/Financiamiento total/interno/Total de endeudamiento no bancario de emisoras/end,Financiamiento total al sector privado no financiero/Financiamiento total/interno/Total de endeudamiento no bancario de emisoras/end/end,Financiamiento total al sector privado no financiero/Financiamiento total/interno/Total de endeudamiento no bancario de emisoras/end/end/end
ds,,,,,,,,,,,,,,,,,,,,,
2024-12-31,1.112390e-08,1.112390e-08,1.108761e-08,1.108761e-08,1.108761e-08,1.108761e-08,0.0,1.108877e-08,-1.167967e-12,3.629510e-11,...,0.000845,0.000845,0.000815,0.000815,0.000815,0.000815,0.006118,0.006118,0.006118,0.006118
2025-03-31,2.029525e-08,2.029525e-08,2.026610e-08,2.026610e-08,2.026610e-08,2.026610e-08,0.0,2.027226e-08,-6.156709e-12,2.914952e-11,...,0.000851,0.000851,0.000715,0.000715,0.000715,0.000715,0.006107,0.006107,0.006107,0.006107
2025-06-30,2.633326e-08,2.633326e-08,2.627971e-08,2.627971e-08,2.627971e-08,2.627971e-08,0.0,2.628178e-08,-2.075968e-12,5.354758e-11,...,0.000849,0.000849,0.000737,0.000737,0.000737,0.000737,0.006042,0.006042,0.006042,0.006042
2025-09-30,2.607215e-08,2.607215e-08,2.601122e-08,2.601122e-08,2.601122e-08,2.601122e-08,0.0,2.601236e-08,-1.142860e-12,6.093701e-11,...,0.000853,0.000853,0.000738,0.000738,0.000738,0.000738,0.006024,0.006024,0.006024,0.006024
2025-12-31,3.025889e-08,3.025889e-08,3.018521e-08,3.018521e-08,3.018521e-08,3.018521e

In [120]:
# Función para procesar los datos de pronóstico
def procesar_pronosticos(reconciled, forecast):
    # Separar unique_id en entidad y concepto
    reconciled['entidad'] = reconciled['unique_id'].str.split('/', expand=True)[0]
    reconciled['idconcepto'] = reconciled['unique_id'].str.split('/').str[-1]
    forecast['entidad'] = forecast['unique_id'].str.split('/', expand=True)[0]
    forecast['idconcepto'] = forecast['unique_id'].str.split('/').str[-1]
    
    # Convertir fechas a formato adecuado
    reconciled['periodo'] = reconciled['ds'].dt.to_period('M').dt.end_time
    forecast['periodo'] = forecast['ds'].dt.to_period('M').dt.end_time
    
    # Crear tabla de comparación
    comparison_df = pd.merge(
        reconciled[['entidad', 'idconcepto', 'periodo', 'AutoARIMA/BottomUp']],
        forecast[['entidad', 'idconcepto', 'periodo', 'AutoARIMA']],
        on=['entidad', 'idconcepto', 'periodo']
    )
    
    # Calcular diferencia porcentual
    comparison_df['Diferencia (%)'] = (
        (comparison_df['AutoARIMA/BottomUp'] - comparison_df['AutoARIMA']) / 
        comparison_df['AutoARIMA']
    ) * 100
    
    return comparison_df, reconciled, forecast
 
# Procesar datos
comparison_df, reconciled_df, forecast_df = procesar_pronosticos(reconciled, forecast)
 
# Exportar tablas clave
comparison_df.to_csv('comparison_data2.csv', index=False)
reconciled_df.to_csv('reconciled_forecast2.csv', index=False)
forecast_df.to_csv('traditional_forecast2.csv', index=False)

# Exportar estructura jerárquica
S_df.reset_index().to_csv('hierarchy_structure2.csv', index=False)
# pd.DataFrame(tags).to_csv('hierarchy_tags2.csv', index=False)
 
# Exportar participación de mercado
def calcular_participacion(df):
    total_por_periodo = df.groupby('periodo')['AutoARIMA/BottomUp'].transform('sum')
    df['Participacion'] = (df['AutoARIMA/BottomUp'] / total_por_periodo) * 100
    return df

market_share_df = reconciled_df.groupby(['entidad', 'periodo']).sum(numeric_only=True).reset_index()
# market_share_df = calcular_participacion(market_share_df.pivot_table(
#     index='periodo', columns='entidad', values='Participacion'
# )).reset_index()

# market_share_df.to_csv('market_share2.csv', index=False)

display(market_share_df)

,entidad,periodo,AutoARIMA,AutoARIMA/BottomUp
0,Financiamiento bancario y principales fuentes ...,2024-12-31 23:59:59.999999999,1.041901e+01,7.198897e+01
1,Financiamiento bancario y principales fuentes ...,2025-03-31 23:59:59.999999999,1.904527e+01,1.326505e+02
2,Financiamiento bancario y principales fuentes ...,2025-06-30 23:59:59.999999999,2.525491e+01,1.753907e+02
3,Financiamiento bancario y principales fuentes ...,2025-09-30 23:59:59.999999999,2.531014e+01,1.755514e+02
4,Financiamiento bancario y principales fuentes ...,2025-12-31 23:59:59.999999999,2.963868e+01,2.054670e+02
...,...,...,...,...
625,Financiamiento total al sector privado no fina...,2031-03-31 23:59:59.999999999,6.640956e+07,6.838404e+07
626,Financiamiento total al sector privado no fina...,2031-06-30 23:59:59.999999999,6.693290e+07,6.908368e+07
627,Financiamiento total al sector privado no fina...,2031-09-30 23:59:59.999999999,6.735769e+07,6.961152e+07
628,Financiamiento total al sector privado no fina...,2031-12-31 23:59:59.999999999,6.805940e+07,7.062370e+07


In [121]:
import numpy as np

# Convertir a numérico, eliminando NaN y valores infinitos
valid_values = comparison_df['Diferencia (%)'].dropna()
valid_values = valid_values[~valid_values.isin([np.inf, -np.inf])]

# Filtrar valores entre -100 y 100
valid_values = valid_values[(valid_values >= -100) & (valid_values <= 100)]

# Calcular la media
mean_value = valid_values.mean() if not valid_values.empty else np.nan

print(mean_value)

-30.097759359483433


In [122]:
display(forecast)

,unique_id,ds,AutoARIMA,entidad,idconcepto,periodo
0,Financiamiento bancario y principales fuentes ...,2024-12-31,6.757680e-83,Financiamiento bancario y principales fuentes ...,Financiamiento bancario y principales fuentes ...,2024-12-31 23:59:59.999999999
1,Financiamiento bancario y principales fuentes ...,2025-03-31,-9.979959e-67,Financiamiento bancario y principales fuentes ...,Financiamiento bancario y principales fuentes ...,2025-03-31 23:59:59.999999999
2,Financiamiento bancario y principales fuentes ...,2025-06-30,-9.979959e-67,Financiamiento bancario y principales fuentes ...,Financiamiento bancario y principales fuentes ...,2025-06-30 23:59:59.999999999
3,Financiamiento bancario y principales fuentes ...,2025-09-30,-9.979959e-67,Financiamiento bancario y principales fuentes ...,Financiamiento bancario y principales fuentes ...,2025-09-30 23:59:59.999999999
4,Financiamiento bancario y principales fuentes ...,2025-12-31,-9.979959e-67,Financiamiento bancario y principales fuentes ...,Financiamiento bancario y principales fuentes ...,2025-12-31 23:59:59.999999999
...,...,...,...,...,...,...
10135,Financiamiento total al sector privado no fina...,2031-03-31,6.805993e+06,Financiamiento total al sector privado no fina...,end,2031-03-31 23:59:59.999999999
10136,Financiamiento total al sector privado no fina...,2031-06-30,6.851973e+06,Financiamiento total al sector privado no fina...,end,2031-06-30 23:59:59.999999999
10137,Financiamiento total al sector privado no fina...,2031-09-30,6.897953e+06,Financiamiento total al sector privado no fina...,end,2031-09-30 23:59:59.999999999
10138,Financiamiento total al sector privado no fina...,2031-12-31,6.943933e+06,Financiamiento total al sector privado no fina...,end,2031-12-31 23:59:59.999999999


In [123]:
# Extraer el código del banco a partir de unique_id.
# Se asume que unique_id tiene el formato "banco_idconcepto", separado por '_'
forecast['entidad'] = forecast['unique_id'].str.split('/').str[0]
reconciled['entidad'] = reconciled['unique_id'].str.split('/').str[0]

# Agrupar por banco y fecha (ds) y sumar los volúmenes de cartera:
trad_summary = forecast.groupby(['entidad', 'ds'])['AutoARIMA'].sum().reset_index()
hier_summary = reconciled.groupby(['entidad', 'ds'])['AutoARIMA/BottomUp'].sum().reset_index()

# Unir las dos tablas de resumen por banco y fecha
summary = pd.merge(trad_summary, hier_summary, on=['entidad', 'ds'], how='outer')

# Renombrar las columnas para mayor claridad
summary = summary.rename(columns={
    'AutoARIMA': 'Pronóstico Tradicional',
    'AutoARIMA/BottomUp': 'Pronóstico Jerárquico'
})

# Opcional: Ordenar por banco y fecha
summary = summary.sort_values(['entidad', 'ds']).reset_index(drop=True)
summary.to_csv('summary.csv', index=False)
# Mostrar la tabla resultante
display(summary)

,entidad,ds,Pronóstico Tradicional,Pronóstico Jerárquico
0,Financiamiento bancario y principales fuentes ...,2024-12-31,1.041901e+01,7.198897e+01
1,Financiamiento bancario y principales fuentes ...,2025-03-31,1.904527e+01,1.326505e+02
2,Financiamiento bancario y principales fuentes ...,2025-06-30,2.525491e+01,1.753907e+02
3,Financiamiento bancario y principales fuentes ...,2025-09-30,2.531014e+01,1.755514e+02
4,Financiamiento bancario y principales fuentes ...,2025-12-31,2.963868e+01,2.054670e+02
...,...,...,...,...
625,Financiamiento total al sector privado no fina...,2031-03-31,6.640956e+07,6.838404e+07
626,Financiamiento total al sector privado no fina...,2031-06-30,6.693290e+07,6.908368e+07
627,Financiamiento total al sector privado no fina...,2031-09-30,6.735769e+07,6.961152e+07
628,Financiamiento total al sector privado no fina...,2031-12-31,6.805940e+07,7.062370e+07


In [124]:
print(tags['level1'].tolist())

['Financiamiento bancario y principales fuentes altern endeud sec privado del país ', 'Financiamiento bancario y principales fuentes altern endeud sec privado del país Bancario', 'Financiamiento bancario y principales fuentes altern endeud sec privado del país Crédito a la vivienda', 'Financiamiento bancario y principales fuentes altern endeud sec privado del país Crédito al consumo', 'Financiamiento bancario y principales fuentes altern endeud sec privado del país Finan emp y pers físicas c_activ empres', 'Financiamiento bancario y principales fuentes altern endeud sec privado del país Financiamiento', 'Financiamiento bancario y principales fuentes altern endeud sec privado del país Industria de la construcción', 'Financiamiento bancario y principales fuentes altern endeud sec privado del país Industria metálica básica', 'Financiamiento bancario y principales fuentes altern endeud sec privado del país No bancario', 'Financiamiento bancario y principales fuentes altern endeud sec priva

In [125]:
columns = ['Financiamiento bancario y principales fuentes altern endeud sec privado del país Crédito al consumo']
# Participación de mercado para el pronóstico tradicional
trad_summary = forecast.groupby(['entidad', 'ds'])['AutoARIMA'].sum().reset_index()
trad_total = trad_summary.groupby('ds')['AutoARIMA'].transform('sum')
trad_summary['Participacion (%)'] = (trad_summary['AutoARIMA'] / trad_total)
participacion_trad = trad_summary.pivot(index='ds', columns='entidad', values='Participacion (%)').reset_index()
participacion_trad.to_csv('participacion_trad2.csv', index=False)

print("Participación de Mercado - Pronóstico Tradicional")
display(participacion_trad)

# Participación de mercado para el pronóstico jerárquico
hier_summary = reconciled.groupby(['entidad', 'ds'])['AutoARIMA/BottomUp'].sum().reset_index()
hier_total = hier_summary.groupby('ds')['AutoARIMA/BottomUp'].transform('sum')
hier_summary['Participacion (%)'] = (hier_summary['AutoARIMA/BottomUp'] / hier_total)
participacion_hier = hier_summary.pivot(index='ds', columns='entidad', values='Participacion (%)').reset_index()
participacion_hier.to_csv('participacion_hier2.csv', index=False)

print("Participación de Mercado - Pronóstico Jerárquico")
display(participacion_hier)


Participación de Mercado - Pronóstico Tradicional


entidad,ds,Financiamiento bancario y principales fuentes altern endeud sec privado del país,Financiamiento bancario y principales fuentes altern endeud sec privado del país Bancario,Financiamiento bancario y principales fuentes altern endeud sec privado del país Crédito a la vivienda,Financiamiento bancario y principales fuentes altern endeud sec privado del país Crédito al consumo,Financiamiento bancario y principales fuentes altern endeud sec privado del país Finan emp y pers físicas c_activ empres,Financiamiento bancario y principales fuentes altern endeud sec privado del país Financiamiento,Financiamiento bancario y principales fuentes altern endeud sec privado del país Industria de la construcción,Financiamiento bancario y principales fuentes altern endeud sec privado del país Industria metálica básica,Financiamiento bancario y principales fuentes altern endeud sec privado del país No bancario,...,Financiamiento bancario y principales fuentes altern endeud sec privado del país Prod metálicos,Financiamiento bancario y principales fuentes altern endeud sec privado del país Servicios y comercio,Financiamiento bancario y principales fuentes altern endeud sec privado del país Uniones de crédito,Financiamiento bancario y principales fuentes altern endeud sec privado del paísFinanciamiento proveniente del extranjer,Financiamiento bancario y principales fuentes altern endeud sec privado del paísProd aliment,Financiamiento bancario y principales fuentes altern endeud sec privado del paísProductos alimenticios,Financiamiento bancario y principales fuentes altern endeud sec privado del paísProductos metálicos,Financiamiento bancario y principales fuentes altern endeud sec privado del paísText,Financiamiento bancario y principales fuentes altern endeud sec privado del paísTextiles,Financiamiento total al sector privado no financiero
0,2024-12-31,1.127943e-08,0.030840,0.071777,0.033626,0.083439,0.326553,0.014000,0.001999,0.106119,...,0.001459,0.055680,0.000392,0.008459,0.001343,0.009615,0.002349,0.000188,0.000187,0.060407
1,2025-03-31,2.043177e-08,0.030878,0.072833,0.034161,0.082798,0.327159,0.013995,0.002015,0.106117,...,0.001459,0.056035,0.000379,0.008128,0.001301,0.009692,0.002387,0.000193,0.000194,0.059693
2,2025-06-30,2.658177e-08,0.031315,0.072057,0.034697,0.083252,0.327291,0.013993,0.002016,0.105710,...,0.001468,0.056453,0.000381,0.008220,0.001293,0.009678,0.002306,0.000194,0.000195,0.059440
3,2025-09-30,2.638016e-08,0.031433,0.072077,0.035393,0.083173,0.327572,0.013999,0.001988,0.105137,...,0.001459,0.057009,0.000396,0.008103,0.001286,0.009724,0.002264,0.000198,0.000199,0.059321
4,2025-12-31,3.065094e-08,0.031403,0.071954,0.036190,0.082411,0.327516,0.014039,0.002003,0.104767,...,0.001461,0.057550,0.000392,0.008068,0.001276,0.009796,0.002269,0.000196,0.000197,0.059716
5,2026-03-31,3.335371e-08,0.031327,0.072760,0.036808,0.081947,0.327422,0.014058,0.002009,0.104455,...,0.001459,0.057913,0.000382,0.008115,0.001264,0.009856,0.002363,0.000194,0.000195,0.059148
6,2026-06-30,3.573424e-08,0.031252,0.072661,0.037567,0.081709,0.327530,0.014076,0.002028,0.104032,...,0.001463,0.058271,0.000387,0.008101,0.001252,0.009916,0.002417,0.000192,0.000193,0.059246
7,2026-09-30,3.539569e-08,0.031135,0.072588,0.038189,0.081474,0.327712,0.014074,0.002014,0.103945,...,0.001457,0.058535,0.000397,0.008059,0.001239,0.009958,0.002435,0.000190,0.000191,0.059063
8,2026-12-31,3.506163e-08,0.031014,0.072262,0.038819,0.081290,0.327749,0.014070,0.001998,0.103784,...,0.001453,0.058784,0.000393,0.008047,0.001226,0.009997,0.002422,0.000188,0.000189,0.059185
9,2027-03-31,3.471471e-08,0.030901,0.072832,0.039391,0.081155,0.327901,0.014068,0.001984,0.103558,...,0.001449,0.059022,0.000385,0.008033,0.001214,0.010038,0.002410,0.000187,0.000187,0.058604


Participación de Mercado - Pronóstico Jerárquico


entidad,ds,Financiamiento bancario y principales fuentes altern endeud sec privado del país,Financiamiento bancario y principales fuentes altern endeud sec privado del país Bancario,Financiamiento bancario y principales fuentes altern endeud sec privado del país Crédito a la vivienda,Financiamiento bancario y principales fuentes altern endeud sec privado del país Crédito al consumo,Financiamiento bancario y principales fuentes altern endeud sec privado del país Finan emp y pers físicas c_activ empres,Financiamiento bancario y principales fuentes altern endeud sec privado del país Financiamiento,Financiamiento bancario y principales fuentes altern endeud sec privado del país Industria de la construcción,Financiamiento bancario y principales fuentes altern endeud sec privado del país Industria metálica básica,Financiamiento bancario y principales fuentes altern endeud sec privado del país No bancario,...,Financiamiento bancario y principales fuentes altern endeud sec privado del país Prod metálicos,Financiamiento bancario y principales fuentes altern endeud sec privado del país Servicios y comercio,Financiamiento bancario y principales fuentes altern endeud sec privado del país Uniones de crédito,Financiamiento bancario y principales fuentes altern endeud sec privado del paísFinanciamiento proveniente del extranjer,Financiamiento bancario y principales fuentes altern endeud sec privado del paísProd aliment,Financiamiento bancario y principales fuentes altern endeud sec privado del paísProductos alimenticios,Financiamiento bancario y principales fuentes altern endeud sec privado del paísProductos metálicos,Financiamiento bancario y principales fuentes altern endeud sec privado del paísText,Financiamiento bancario y principales fuentes altern endeud sec privado del paísTextiles,Financiamiento total al sector privado no financiero
0,2024-12-31,7.786731e-08,0.030814,0.071691,0.033594,0.083367,0.326452,0.013990,0.001995,0.106038,...,0.001458,0.055680,0.000392,0.008452,0.001342,0.009606,0.002347,0.000188,0.000187,0.060328
1,2025-03-31,1.420668e-07,0.030826,0.072721,0.034072,0.082658,0.326971,0.013968,0.002010,0.105924,...,0.001457,0.056010,0.000378,0.008114,0.001298,0.009676,0.002383,0.000193,0.000193,0.059604
2,2025-06-30,1.843328e-07,0.031268,0.071950,0.034646,0.083129,0.327037,0.013965,0.002010,0.105451,...,0.001466,0.056454,0.000381,0.008207,0.001291,0.009664,0.002302,0.000194,0.000194,0.059260
3,2025-09-30,1.825051e-07,0.031352,0.071914,0.035297,0.082960,0.327090,0.013954,0.001980,0.104955,...,0.001456,0.056972,0.000395,0.008082,0.001282,0.009699,0.002258,0.000197,0.000198,0.059191
4,2025-12-31,2.118123e-07,0.031304,0.071724,0.036086,0.082150,0.327083,0.013984,0.001995,0.104618,...,0.001456,0.057523,0.000391,0.008043,0.001272,0.009765,0.002261,0.000195,0.000196,0.059748
5,2026-03-31,2.310366e-07,0.031218,0.072572,0.036681,0.081663,0.327032,0.013996,0.002002,0.104323,...,0.001454,0.057894,0.000381,0.008087,0.001260,0.009822,0.002355,0.000194,0.000195,0.059045
6,2026-06-30,2.468630e-07,0.031125,0.072420,0.037434,0.081376,0.327019,0.014004,0.002023,0.103937,...,0.001457,0.058243,0.000385,0.008068,0.001247,0.009876,0.002407,0.000192,0.000193,0.059188
7,2026-09-30,2.442279e-07,0.030994,0.072331,0.038040,0.081106,0.327125,0.013994,0.002009,0.103760,...,0.001451,0.058505,0.000395,0.008023,0.001234,0.009913,0.002424,0.000190,0.000191,0.059081
8,2026-12-31,2.416457e-07,0.030865,0.071962,0.038670,0.080900,0.327111,0.013984,0.001994,0.103523,...,0.001446,0.058761,0.000391,0.008008,0.001220,0.009949,0.002410,0.000188,0.000188,0.059452
9,2027-03-31,2.391611e-07,0.030750,0.072604,0.039234,0.080758,0.327264,0.013979,0.001980,0.103267,...,0.001442,0.059014,0.000383,0.007994,0.001208,0.009989,0.002398,0.000186,0.000187,0.058706
